# Graph Mode Computations
In previous section of the course we investigate the properties and characteristics of **Graph Mode** and **Eager Mode** in TensorFlow. We saw that although most of times they behave same, there are cases where they behave differently like writing `print` statements or debugging. \
In this assignment we will investigate other cases:

In [1]:
import tensorflow as tf
import numpy as np

## 1. Lists
Python list is poorly supported in graph mode. In particular, when the list is modified inside or outside the `@tf.function` method.\
**Run the next two cells to see the unexpected behaviour.**


In [2]:
l = []

@tf.function
def f(x):
    for i in x:
        l.append(i + 1)

In [3]:
f(tf.constant([1, 2, 3]))
print(l)

[<tf.Tensor 'while/add:0' shape=() dtype=int32>]


Let's rewrite this function with the use of `TensorArray` which is a data structure for dynamic arrays.
use its `write` and `stack` method to rewrite the above function.\
https://www.tensorflow.org/api_docs/python/tf/TensorArray#write

https://www.tensorflow.org/api_docs/python/tf/TensorArray#stack

In [4]:
@tf.function
def f(x):
    ta = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    for i in range(len(x)):
        ta = ta.write(i, i+1)
    return ta.stack()

In [5]:
a = f(tf.constant([1, 2, 3]))
a

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

## 2. Tensor Objects and Numpy Arrays
With eager execution, Numpy operations can take tf.Tensor as parameters.\
run the following cells

In [6]:
def np_multiply(a, b):
    return np.multiply(a, b)

In [7]:
a_tensor = tf.constant([5])
b_tensor = tf.constant([10])
np_multiply(a_tensor, b_tensor)

array([50], dtype=int32)

What about vice versa?\
Write an eager function that gets two numpy array and by using `tf.math.multiply` multiply these arrays.

In [8]:
# @tf.function
def tf_multiply(a, b):
    return tf.math.multiply(a,b)

In [9]:
a_array = np.array([5])
b_array = np.array([10])
tf_multiply(a_array, b_array)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([50])>

now convert there two functions into Graph functions and run again:

In [10]:
np_multiply_tf_func = tf.function(tf_multiply)
tf_multiply_tf_func = tf.function(np_multiply)

In [11]:
np_multiply_tf_func(a_tensor, b_tensor)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([50], dtype=int32)>

As you noticed we can't use `numpy` operations inside graph functions and we must use their `TensorFlow` equivalents.\
Also TensorFlow has a new API which implemented the numpy functions:
https://www.tensorflow.org/guide/tf_numpy

In [12]:
tf_multiply_tf_func(a_array, b_array)

NotImplementedError: in user code:

    File "<ipython-input-6-04bf7d2dffda>", line 2, in np_multiply  *
        return np.multiply(a, b)

    NotImplementedError: Cannot convert a symbolic tf.Tensor (a:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.


## 3. Tensor Assignment
Let's write an eager function that gets a one dimensional Tensor and assing `1` to its first element

In [13]:
def assign_one(x):
    x[1] = 1
    return x

In [14]:
x = tf.constant([5, 6, 7])
assign_one(x)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

You can see that TensorFlow `Tensor` object does not support item assignment in contrast with numpy array or python lists

But there is a way to do this in eager mode:
1. create a numpy array from the tensor by calling `tensor.numpy()
2. assign the value to numpy array
3. create a constant tensor from the numpy array

let's write it

In [15]:
def assign_one_new(x):
    x_numpy = x.numpy()
    x_numpy[0] = 1
    return tf.constant(x_numpy)

In [16]:
x = tf.constant([5, 6, 7])
assign_one_new(x)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 6, 7], dtype=int32)>

Now convert this function to a Graph function

In [17]:
assign_one_graph = tf.function(assign_one_new)

In [18]:
x = tf.constant([5, 6, 7])
assign_one_graph(x)

AttributeError: in user code:

    File "<ipython-input-15-08843867c801>", line 2, in assign_one_new  *
        x_numpy = x.numpy()

    AttributeError: 'SymbolicTensor' object has no attribute 'numpy'


You can see that there is no way to assign a value to a `Tensor` in graph mode, you should consider it in your implementations.
You can use `tf.Variable` instead of Tensors for assignments:
https://www.tensorflow.org/api_docs/python/tf/Variable#assign

## Conclusion
In this assignment we have investigated some differences of Graph mode and Eager mode in TensorFlow, specially we saw that working with Python Lists and Numpy Arrays have some difficulties with Graph Mode